In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)

# Demand Elasticity

This example demonstrates how demand elasticity can be modelled in PyPSA, using single node capacity expansion model in the style of [model.energy](https://model.energy).

See [Brown, Neumann, Riepin (2025)](https://doi.org/10.1016/j.eneco.2025.108483) for more details.

## Preparations

We start by loading packages, example networks and creating a utility function to retrieve price duration curve.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pypsa

plt.style.use("bmh")

In [ ]:
def get_price_duration(n: pypsa.Network, bus: str = "electricity") -> pd.Series:
    s = (
        n.buses_t.marginal_price[bus]
        .sort_values(ascending=False)
        .reset_index(drop=True)
    )
    s.index = np.arange(0, 100, 100 / len(s.index))
    return s

In [ ]:
n = pypsa.examples.model_energy()
n.remove("Load", "demand")
n.remove("Generator", "load shedding")

To save some computation time, we will sample just every fifth day of the year. Each day is considered at 3-hourly resolution, so we will have 8 snapshots per representative day.

In [ ]:
selected = n.snapshots.normalize().unique()[::5]
snapshots = n.snapshots[n.snapshots.normalize().isin(selected)]
n.set_snapshots(snapshots)
n.snapshot_weightings[["objective", "generators"]] *= 5

## Perfectly inelastic demand

Most commonly, capacity expansion models would prescribe a perfectly inelastic demand via the `p_set` attribute, e.g. 100 MW.

The utility drawn from this consumption is effectively infinite. The model has to find a way to satisfy it. Otherwise, the model is infeasible.

In [ ]:
n.add("Load", "demand", bus="electricity", p_set=100)

In [ ]:
n.optimize()

Market clearing prices can spike to extreme values in few hours of the year, while remaining close to zero for a majority of time.

In [ ]:
fig, ax = plt.subplots()
get_price_duration(n).plot(
    ax=ax,
    ylabel="Clearing Price [€/MWh]",
    xlabel="Fraction of Time [%]",
    label="default",
    legend=True,
)

In [ ]:
capacities = n.statistics.optimal_capacity(round=2).to_frame("inelastic")
capacities

## Perfectly inelastic demand up to VOLL

One way to avoid the price spikes is to model demand as perfectly inelastic up to a pre-defined value of lost load (VOLL).

Effectively, this is defined by a utility function $U(d) = Vd$ with a constant value $V$ for consumption $d\in[0,D]$, for instance 1000 €/MWh.

The demand curve is a step function. It is perfectly inelastic up to a price of $V$ at which point it is perfectly elastic.

When we make the substitution $d=D-g$, we see that we can model the VOLL case with a load shedding generator with marginal costs of $V=1000$ €/MWh, omitting the constant term $VD$.

$U(d) = Vd$

$U(d) = VD - Vg$

Note that the objective sense of PyPSA is to minimise costs in order to maximise utility, so any costs have a postive sign and utility gains have a negative sign in the objective.

In [ ]:
n.add(
    "Generator",
    "load-shedding",
    bus="electricity",
    carrier="load",
    marginal_cost=1000,
    p_nom=100,
)

In [ ]:
n.optimize()

Now, the peak price is capped at 2000 €/MWh:

In [ ]:
get_price_duration(n).plot(ax=ax, label="VOLL", legend=True)
fig

This results in some changes in the cost-optimal capacity mix, in particular in terms of backup capacities.

In [ ]:
capacities["VOLL"] = n.statistics.optimal_capacity(round=2)
capacities

## Linear demand curve

In reality, electricity demand is at least partially elastic. Consumers would use less electricity if its more expensive, or would use more electricity if prices are low. 

For a linear demand curve $p = a - bd$, where $p$ is the price, the utility is quadratic: $U(d) = ad - 0.5 b d^2$

For a choice of $a=2000$ and $b=20$, the demand curve looks like this:

In [ ]:
x = np.linspace(0, 100, 200)
plt.figure(figsize=(8, 4))
plt.plot(x, 2000 - 20 * x)
plt.xlabel("Demand (MW)")
plt.ylabel("Price (€/MWh)")

That means, for instance, at a price of 1000 €/MWh, the demand would be only 50 MW. At a price of 400 €/MWh, 80 MW. And so on.

Applying the same substition $d = a/b - g$ ($a/b$ yields the maximum consumption at the zero point), turns 

$U(d) = ad - 0.5 b d^2$

into 

$U(g) = \frac{a^2}{2b} - 0.5 b g^2$

which represents a load shedding generator with quadratic marginal cost $b/2$, again omitting the constant term from the objective.

Due to the quadratic terms in the objective function, this addition turns the model into a quadratic problem (QP).

In [ ]:
n.remove("Generator", "load-shedding")

n.add(
    "Generator",
    "load-shedding",
    bus="electricity",
    carrier="load",
    marginal_cost_quadratic=20 / 2,
    p_nom=100,
)

In [ ]:
n.optimize(solver_name="highs")

The price duration curve is considerably smoother with less extreme prices and fewer zero-price hours:

In [ ]:
get_price_duration(n).plot(ax=ax, label="linear-elastic", legend=True)
fig

Also, the optimised capacity mix is drastically different. The model cuts down on balancing technologies and rather curtails a lot demand instead. 

In [ ]:
capacities["linear-elastic"] = n.statistics.optimal_capacity(round=2)
capacities

The drawback and explanation here is that the linear demand curve becomes unrealistically elastic at higher prices.

## Partial demand elasticity

It is also possible to mix different demand modelling approaches. For instance, keeping 80% of demand perfectly inelastic, while modelling 20% with a linear demand curve.

This just requires adjusting the capacity and cost terms of the load shedding generator (adjusts the slope of the linear demand curve)

In [ ]:
n.generators.loc["load-shedding", "p_nom_max"] *= 0.2
n.generators.loc["load-shedding", "marginal_cost_quadratic"] /= 0.2

## Piecewise-linear demand curve

It is also possible to model a set of piecewise linear demand curves, e.g. to approximate a log-log demand curve ($\ln p = a - b \ln d$), without much modification.

For details see [Brown, Neumann, Riepin (2025)](https://doi.org/10.1016/j.eneco.2025.108483), Section 3.2 and Appendix A. 